In [12]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


In [13]:
# Load the dictionary from the pickle file
with open('signs_dict4.pkl', 'rb') as f:
    data = pkl.load(f)

In [14]:
X_train = np.array(data['X_train'])
X_test = np.array(data['X_test'])
y_train = np.array(data['y_train'])
y_test = np.array(data['y_test'])

In [15]:
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

(1085, 4)
(1085, 365, 324, 3)
(108, 4)
(108, 365, 324, 3)


In [16]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

In [17]:
# Define the model
model = keras.Sequential([
    layers.Input(shape=(365, 324, 3)),
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Add dropout for regularization
    layers.Dense(4, activation='softmax')  # 4 classes
])

In [18]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ (None, 365, 324, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 363, 322, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 181, 161, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 179, 159, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 89, 79, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 87, 77, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 43, 38, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 41, 36, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 20, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 92160)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    23,593,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,982,660 (91.49 MB)

 Trainable params: 23,982,660 (91.49 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model with data augmentation
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 24s 673ms/step - accuracy: 0.2596 - loss: 2.3220 - val_accuracy: 0.3981 - val_loss: 1.2930
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 22s 660ms/step - accuracy: 0.4392 - loss: 1.2574 - val_accuracy: 0.4815 - val_loss: 1.1141
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 669ms/step - accuracy: 0.5014 - loss: 1.1211 - val_accuracy: 0.5185 - val_loss: 1.1125
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 22s 639ms/step - accuracy: 0.5472 - loss: 1.1306 - val_accuracy: 0.5741 - val_loss: 1.0156
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 22s 634ms/step - accuracy: 0.6116 - loss: 0.9868 - val_accuracy: 0.6111 - val_loss: 1.0458
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 21s 627ms/step - accuracy: 0.6444 - loss: 0.9099 - val_accuracy: 0.6111 - val_loss: 0.9859
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 22s 631ms/step - accuracy: 0.6714 - loss: 0.8446 - val_accuracy: 0.6481 - val_loss: 0.8868
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 22s 633ms/step - accuracy: 0.6922 - loss: 0.7359 - val_accu

In [21]:
# Print classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=['crosswalk', 'speedlimit', 'stop', 'trafficlight']))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step
              precision    recall  f1-score   support

   crosswalk       0.79      0.67      0.72        33
  speedlimit       0.65      0.80      0.72        30
        stop       0.83      0.76      0.79        25
trafficlight       0.75      0.75      0.75        20

    accuracy                           0.74       108
   macro avg       0.75      0.74      0.74       108
weighted avg       0.75      0.74      0.74       108



In [22]:
# Save the model to a file
model.save('traffic_signs_model4.keras')